# Obtenção e tratamento de dados


## Obtenção e carregamento de dados


In [ ]:
#importação de bibliotecas
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
#Carregamento de dados limpos da pasta ../data/
df = pd.read_csv('../Data/dataset.csv')

## Informações sobre o dataset


In [ ]:
#Exibição dos dados
df.head()

In [ ]:
#Exibição de informações sobre os dados
df.info()

In [ ]:
#Exibição de estatísticas descritivas
df.describe()

## Ajuste de tipos


In [ ]:
#Matricula
df['dt_matricula'] = pd.to_datetime(df['dt_matricula'])
#Saida
df['dt_saida'] = pd.to_datetime(df['dt_saida'])
#Primeiro login
df['primeiro_login'] = pd.to_datetime(df['primeiro_login'])
#Ultimo login
df['ultimo_login'] = pd.to_datetime(df['ultimo_login'])

#Idade
df['idade'] = df['idade'].astype('int64')
#Total de logins
df['total_logins'] = df['total_logins'].astype('int64')
#Dias desde o primeiro login
df['dias_desde_primeiro_login'] = df['dias_desde_primeiro_login'].astype('int64')
# Quantidade Mestrado
df['qtd_professores_mestrado'] = df['qtd_professores_mestrado'].astype('int64')
# Quantidade Doutorado
df['qtd_professores_doutorado'] = df['qtd_professores_doutorado'].astype('int64')
# Quantidade Especialização
df['qtd_professores_especialista'] = df['qtd_professores_especialista'].astype('int64')
# Quantidade Graduado
df['qtd_professores_graduado'] = df['qtd_professores_graduado'].astype('int64')
# Quantidade de professores Masculino
df['qtd_professores_M'] = df['qtd_professores_M'].astype('int64')
# Quantidade de professores Feminino
df['qtd_professores_F'] = df['qtd_professores_F'].astype('int64')
# Menor idade de um professor
df['idade_min_professores'] = df['idade_min_professores'].astype('int64')
# Maior idade de um professor
df['idade_max_professores'] = df['idade_max_professores'].astype('int64')

#Data de registro para string
df['dt_registro'] = df['dt_registro'].astype(object)
#Data de inauguração do curso
df['dt_inauguracao_curso'] = df['dt_inauguracao_curso'].astype(object)

#Exibição de informações sobre os dados
df.info()
#Exibição de estatísticas descritivas
df.head()


## Tratamento de valores ausentes


In [ ]:
#preenche com o texto "Abandono" os valores nulos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'] = df['categoria_motivo_cancelamento'].fillna('Abandono').where(df['evasao'] == 1, 'NA')

#Exibe os valores unicos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'].unique()

Exibição dos dados


In [ ]:
#Exibe as informações sobre os dados
df.info()
#Exibe as estatísticas descritivas
df.head()

# Estatísticas (Regressão logística) - Construção do modelo

## Amostra dos dados

In [ ]:
df.head()

## Codificação de variáveis categóricas e numéricas

In [ ]:
# Exclua as colunas que não serão usadas
df = df.drop(['cd_aluno','nome', 'curso','cidade','titulacao', 'estado', 'semestre_atual','dias_matriculado_semestre','total_logins','qtd_aprovacoes','valor_mensalidade_bruto', 'percentual_desconto','dias_desde_primeiro_login', 'dt_registro', 'cd_curso', 'dt_inauguracao_curso', 'categoria_motivo_cancelamento','situacao','dt_matricula', 'dt_saida', 'primeiro_login', 'ultimo_login', 'idade_min_professores', 'idade_max_professores','qtd_professores','qtd_professores_F'], axis=1)

In [ ]:
# Defina as colunas numéricas e categóricas
num_cols = ['idade', 'evasao', 'valor_mensalidade_liquido', 'qtd_mensalidade_paga_atraso', 'qtd_semestres_curso', 'qtd_reprovacoes', 'coeficiente_rendimento',  'media_acessos_por_dia', 'semestres_existencia_curso', 'qtd_professores_mestrado', 'qtd_professores_doutorado', 'qtd_professores_especialista', 'qtd_professores_graduado', 'qtd_professores_M',  'idade_media_professores']
cat_cols = ['genero', 'ds_ensino_medio', 'ds_ingresso', ]

#Criar um dataframe de cópia
df_original = df.copy()


# Defina a ordem desejada para cada variável categórica
genero_ordem = ['NF', 'M', 'F']
ds_ensino_medio_ordem = ['Não informado', 'Escola Pública', 'Escola Privada']
ds_ingresso_ordem = ['Reingresso', 'ENEM', 'Segunda Graduação', 'Transferência', 'Vestibular']

# Reordene as categorias
df['genero'] = df['genero'].astype('category').cat.reorder_categories(genero_ordem, ordered=True)
df['ds_ensino_medio'] = df['ds_ensino_medio'].astype('category').cat.reorder_categories(ds_ensino_medio_ordem, ordered=True)
df['ds_ingresso'] = df['ds_ingresso'].astype('category').cat.reorder_categories(ds_ingresso_ordem, ordered=True)

# Realize a codificação one-hot e remova a primeira variável fictícia para cada variável categórica
df = pd.get_dummies(df, columns=['genero', 'ds_ensino_medio', 'ds_ingresso'], drop_first=True)

# Converta as colunas booleanas em inteiros
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int)

df.info()
df.head()

## Definição de variáveis independentes e dependentes


In [ ]:
X = df.drop(['evasao'], axis=1)
y = df['evasao']

## Adição de constante para ajuste do modelo

In [ ]:
X = sm.add_constant(X)

## Verificar o fator de inflação da variância (VIF)

In [ ]:
# Calcule o VIF para cada variável independente
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.sort_values('VIF Factor', ascending=False).head(60)

## Ajuste do modelo

In [ ]:
modelo = sm.Logit(y, X)
resultado = modelo.fit()

##  Resultados do modelo

In [ ]:
print(resultado.summary())

### Odds ratio

In [ ]:

# Armazenando as informações do cabeçalho
header_info = {
    'Dep. Variable': resultado.model.endog_names,
    'No. Observations': resultado.nobs,
    'Model': 'Logit',
    'Df Residuals': resultado.df_resid,
    'Method': 'MLE',
    'Df Model': resultado.df_model,
    'Date': 'Tue, 11 Jun 2024',
    'Pseudo R-squ.': resultado.prsquared,
    'Time': '10:41:29',
    'Log-Likelihood': resultado.llf,
    'converged': resultado.mle_retvals['converged'],
    'LL-Null': resultado.llnull,
    'Covariance Type': 'nonrobust',
    'LLR p-value': resultado.llr_pvalue,
}

# Imprimindo as informações do cabeçalho
for key, value in header_info.items():
    print(f'{key}: {value}')

# Calculando os odds-ratios
odds_ratios = np.exp(resultado.params)

# Criando um DataFrame para o sumário estendido
summary_df = pd.DataFrame()
summary_df['coef'] = resultado.params
summary_df['std err'] = resultado.bse
summary_df['z'] = resultado.tvalues
summary_df['P>|z|'] = resultado.pvalues
summary_df['[0.025'] = resultado.conf_int()[0]
summary_df['0.975]'] = resultado.conf_int()[1]
summary_df['odds_ratio'] = odds_ratios

# Imprimindo o sumário estendido
summary_df